## Threading Demo.

This notebook demonstrates how to carry out parameter scans using threading.
It also performs some basic analysis based on a parameter scan of the initial
electron angle.

In [ ]:
%run config.py

In [ ]:
from ford1991 import Ford1991Solver
from lorentz import LorentzSolver
from qtnm_fields import BathTubField
from scipy.constants import c, electron_mass as me, elementary_charge as qe, mu_0 as mu0
from trace import trace_theta
import concurrent.futures
from functools import partial
import copy

In [ ]:
# Initial kinetic energy (eV)
T = 18600
# Rel. gamma
gamma_rel = T * qe / (me*c**2) + 1
# (v/c)^2
beta_sq = 1 - 1 / gamma_rel**2
# Initial electron speed
v0 = np.sqrt(beta_sq) * c
# Background B-field (T)
B0 = np.array([0, 0, 1.0])

# Distance between coils
coil_distance = 0.5
# Coil radius
r_coil = 0.03

In [ ]:
# Some helper functions
def current_from_b(b):
    return 2.0 * r_coil * b / mu0

def on_axis_field(z, current):
    return mu0 * current * r_coil**2 / (2.0 * (r_coil**2 + z**2)**(1.5))

def zfromb(b, current):
    tmp = (mu0 * r_coil**2 * current / 2.0 / b)**(2.0/3.0)
    if tmp < r_coil**2:
        return 0.0
    return np.sqrt(tmp - r_coil**2)

In [ ]:
# Set up coil calculation. Coil in X-Y plane, centred on (0,0)
Ny = 51
Nz = 101

# Require an 86 degree limit, assuming a mid-point field of 1T
deltaB = np.cos(np.deg2rad(86))**2
print('Delta B = %.4f' % deltaB)

# Calculate current such that peak B = 4mT
current = 2.0 * deltaB * r_coil / mu0
print('Total current = %.4f' % current)

# Set up a QTNM like field, with coils at +/- 3cm
zc1 = -0.5*coil_distance
zc2 = 0.5*coil_distance
qtnm_bottle = BathTubField(radius=r_coil, current=current, Z1 = zc1, Z2 = zc2, background=B0)
print('Max field perturbation = %.4E' % (qtnm_bottle.evaluate_field(0,0,zc1)[2] - 1))

In [ ]:
# Set-up Ford solver
ford_solver = Ford1991Solver(calc_b_field=qtnm_bottle.evaluate_field)

In [ ]:
# Set up partial function
trace_theta_p = partial(trace_theta, solver=ford_solver, v0=v0, nrot=1e4)

In [ ]:
# Test trace for theta = -1 degree
res = trace_theta_p(-np.deg2rad(1.0))

In [ ]:
# Calculate field on 2D slice
ygrid = np.linspace(-0.05,0.05,Ny)
zgrid = np.linspace(1.1*zc1,1.1*zc2,Nz)

Y, Z = np.meshgrid(ygrid, zgrid)

bx = np.zeros_like(Y)
by = np.zeros_like(Y)
bz = np.zeros_like(Y)

# For plotting purposes subtract background back off
for i in range(Nz):
    for j in range(Ny):
        x = 0.0
        y = ygrid[j]
        z = zgrid[i]
        
        bx[i,j], by[i,j], bz[i,j] = qtnm_bottle.evaluate_field(x, y, z) - B0

In [ ]:
x = res.y[0]
y = res.y[1]
z = res.y[2]
incr = 10 # Plot every 10th point of trace

plt.streamplot(Y, Z, by, bz, color="blue", linewidth=0.1, density=2)
plt.plot(-r_coil,zc1, markersize=3, marker='o', color='orange', alpha=0.75)
plt.plot(r_coil,zc1, markersize=3, marker='o', color='orange', alpha=0.75)
plt.plot(-r_coil,zc2, markersize=3, marker='o', color='orange', alpha=0.75)
plt.plot(r_coil,zc2, markersize=3, marker='o', color='orange', alpha=0.75)
ax = plt.gca()

xtrace = x[::incr]
ztrace = z[::incr]
plt.plot(xtrace, ztrace, alpha = 0.25, color='red')

plt.xlim(ygrid[0], ygrid[-1])
plt.ylim(zgrid[0], zgrid[-1])
plt.xlabel(r'$z(m)$')
plt.ylabel(r'$y(m)$')
plt.tight_layout()

In [ ]:
# Check what sort of range of vertical field the electron has experienced
plt.plot(zgrid[:], bz[:,25])
plt.axvspan(np.min(res.y[2]), np.max(res.y[2]), color='blue', alpha=0.5)

In [ ]:
# Does this match expectation?
# Note our values of theta defined relative to beam
# Check turning point B-field
t_initial = 0.5 * np.pi - np.deg2rad(1.0) # 89 degrees wrt to B-field
b_initial = qtnm_bottle.evaluate_field(0,0,0)[2]
print(b_initial)
b_turning = np.cos(t_initial)**2
analytic_turning = (1.0 + b_turning) * b_initial
print(analytic_turning)
numerical_turning = qtnm_bottle.evaluate_field(0,0,np.min(res.y[2]))[2]
print(numerical_turning)
print(np.abs(numerical_turning - analytic_turning) / analytic_turning)

In [ ]:
# Range of theta values
theta_deg = np.arange(0,5,0.1)
theta = np.deg2rad(-theta_deg)

In [ ]:
%%time
with concurrent.futures.ProcessPoolExecutor(max_workers = 4) as executor:
    future = executor.map(trace_theta_p, theta)
results = list(future)

In [ ]:
# Make a back-up copy
results_copy = copy.deepcopy(results)

In [ ]:
# Find minimum height for each trace
zmax = []
for r in results:
    zmax.append(np.min(r.y[2]))

In [ ]:
plt.plot(theta_deg, zmax)
plt.ylim(-0.3, 0.025)
plt.xlim(0,5)
plt.xlabel(r'$\theta \mathrm{(deg.)}$')
plt.ylabel(r'$\mathrm{Max.\; Displ.\;(m)}$')
plt.axhline(-0.25, color='black', linestyle='--')
plt.axvline(4.0, color='black', linestyle='--')
plt.title(r'$\mathrm{R_{coil} = 3cm,\;B_{coil} = 4.9mT}$')
plt.tight_layout()

In [ ]:
zmax_clip = np.array(zmax).clip(min=-0.25)
b_turning_numerical = []
for z in zmax_clip:
    b_turning_numerical.append(qtnm_bottle.evaluate_field(0,0,np.min(z))[2])
b_turning_numerical = np.array(b_turning_numerical)

b_turning_theory = []
for t in theta:
    cos2 = np.sin(t)**2
    b_turning_theory.append((1.0 + cos2) * b_initial)
b_turning_theory = np.array(b_turning_theory)

In [ ]:
plt.plot(theta_deg, b_turning_numerical, label='Numerical')
plt.plot(theta_deg, b_turning_theory, label='Theory', linestyle='--')

In [ ]:
theta_mod = 90 - theta_deg
fig, ax = plt.subplots(1,2, figsize=[12,6])
ax[0].plot(theta_mod, zmax)
ax[0].set_ylim(-0.3, 0.025)
ax[0].set_xlim(90,85)
ax[0].set_xlabel(r'$\theta \mathrm{(deg.)}$')
ax[0].set_ylabel(r'$\mathrm{Max.\; Displ.\;(m)}$')
ax[0].axhline(-0.25, color='black', linestyle='--')
ax[0].axvline(86.0, color='black', linestyle='--')
plt.suptitle(r'$\mathrm{R_{coil} = 3cm,\;B_{coil} = 4.9mT,\;L_{Solen.} = 50cm}$')
ax[1].plot(theta_mod, b_turning_numerical, label='Numerical')
ax[1].plot(theta_mod, b_turning_theory, label='Theory', linestyle='--')
ax[1].set_xlabel(r'$\theta \mathrm{(deg.)}$')
ax[1].set_ylabel(r'$\mathrm{Max.\; B.\;(T)}$')
ax[1].set_xlim(90,85)
plt.legend()
plt.tight_layout()
plt.savefig('Theory_Test_86deg.png')

In [ ]:
b_rms = []
# Loop over results
for ir, r in enumerate(results):
    bmag2_tot = 0.0
    # Only want to record up to turning point
    for i in np.arange(len(r.t)):
        bmag2_tot += qtnm_bottle.evaluate_field_magnitude(r.y[0][i],r.y[1][i],r.y[2][i])**2
        if r.y[2][i] < -0.25 or r.y[5][i] > 0:
            print('Found turning point:', ir, i, r.y[2][i], r.y[5][i])
            b_rms.append(np.sqrt(bmag2_tot / (i + 1)))
            break
        if i == len(r.t) - 1:
            print(ir, i, r.y[2][i], r.y[5][i], 'reached end!')
            b_rms.append(np.sqrt(bmag2_tot / len(r.t))) 

In [ ]:
plt.plot(theta_mod, np.array(b_rms) - 1)
plt.ylim(bottom=0)
plt.yscale('log')
plt.ylim(1e-5,2e-3)
plt.xlim(90,85)
plt.title('RMS Magnetic field along Electron Path')
plt.ylabel(r'$\sqrt{\widebar{B^2}} - 1$')
plt.xlabel(r'$\theta \mathrm{(deg.)}$')
plt.tight_layout()
plt.savefig('BRMS.png')

In [ ]:
# Set up partial function using Lorentz solver to compare performance
lorentz_solver = LorentzSolver(calc_b_field=qtnm_bottle.evaluate_field)
trace_theta_lorentz = partial(trace_theta, solver=lorentz_solver, v0=v0, nrot=1e4)

In [ ]:
%%time
with concurrent.futures.ProcessPoolExecutor(max_workers = 4) as executor:
    future_lorentz = executor.map(trace_theta_lorentz, theta)
results_lorentz = list(future_lorentz)